In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\m3irs\\OneDrive\\Desktop\\2021DataScience\\2023Datascience\\Project\\wine_quality_project\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'c:\\Users\\m3irs\\OneDrive\\Desktop\\2021DataScience\\2023Datascience\\Project\\wine_quality_project'

In [5]:
#Before using the mlfow trcking uri in notebook execute below command 

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Irshad-ml/wine-quality.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] ="Irshad-ml"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3acde6a67d36fe512914bc2abe1603b13fd958e3"


In [6]:
os.environ["MLFLOW_TRACKING_URI"]

'https://dagshub.com/Irshad-ml/wine-quality.mlflow'

In [7]:
from src.mlproject.logger import logging

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    metric_file_name:Path
    all_params:dict
    target_column:str
    mlflow_uri:str
    
    

In [9]:
from src.mlproject.constants import constant
from src.mlproject.utils.common import read_yaml,create_directories,save_json


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=constant.CONFIG_FILE_PATH,
        params_filepath=constant.PARAMS_FILE_PATH,
        schema_filepath=constant.SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = os.environ["MLFLOW_TRACKING_URI"],
        )
        
        
        return model_evaluation_config
        

In [10]:
print(f"config_yaml file  path:  {constant.CONFIG_FILE_PATH}")
print(f"param_yaml  file  path: {constant.PARAMS_FILE_PATH}")
print(f"schema_yaml file  path: {constant.SCHEMA_FILE_PATH}")

config_yaml file  path:  config\config.yaml
param_yaml  file  path: params.yaml
schema_yaml file  path: schema.yaml


In [11]:
read_yaml(constant.CONFIG_FILE_PATH)

config\config.yaml


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Irshad-ml/Dataset-Collection/raw/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_path': 'artifacts/model_trainer/model.joblib', 'metr

In [12]:
config_file_path = constant.CONFIG_FILE_PATH

config_file_path

WindowsPath('config/config.yaml')

In [13]:
read_yaml(config_file_path)

config\config.yaml


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Irshad-ml/Dataset-Collection/raw/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_path': 'artifacts/model_trainer/model.joblib', 'metr

In [14]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [17]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig) :
        self.config = config
        
        
    def eval_metrics(self,actual_value,pred_value):
        rmse = np.sqrt(mean_squared_error(actual_value,pred_value))
        mae  = mean_absolute_error(actual_value,pred_value)
        r2 = r2_score(actual_value,pred_value)
        return rmse,mae,r2    
    
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print("URl tracking",tracking_url_type_store)
        
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            
            (rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)
            
            #saving metrics as local
            
            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2_score",r2)
            

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                logging.info("")
            else:
                mlflow.sklearn.log_model(model, "model")
        
        
        


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

config\config.yaml
params.yaml
schema.yaml
URl tracking https


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/10/09 15:59:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 6
Created version '6' of model 'ElasticnetModel'.
